# Simulating Data with NumPy
In this tutorial, I will briefly outline how to simulate data using Pandas and NumPy. I am including this because I think it is a valuable skill. It is particularly valuable for testing new code you have written. I often write short simulations all the time to test new functions in *zEpid*. So, I thought it would be useful to include a quick tutorial

## Considerations and Caveats
When simulating data, it is important to understand your purpose. You can be conducting a simulation study to demonstrate that you method is better than other methods. That's great and this tutorial will be a useful start for you. However, that is not my focus in this tutorial. My focus is on how to do it in Python with the end purpose being simulation data to experiment with code. When creating simulations to demonstrate your method, you will need to consider sample size, number of simulations to conduct, and how exactly variables are related. Furthermore, thought needs to be put into the simulation to make it reflective of real data (for truly informative simulation studies). 

I am throwing those concerns out and focusing only on the practicalities of simulating data in Python. This tutorial may be useful after you determine the sample size, number of simulations, and your data generating mechanism. For an introduction to conducting thoughtful simulation studies, I would recommend the following paper as a starting point:

Burton, A., Altman, D. G., Royston, P., & Holder, R. L. (2006). The design of simulation studies in medical statistics. SIM, 25(24), 4279-4292.

## Simulating Variables
Now that I have my warning out of the way, let's simulate some data! First we will need a data generating mechanism. We will used the following distributions for the following variables

$W \sim Binomial(1, 0.5)$

$A \sim Binomial(1, 0.05 + 0.45*W)$

To generate the data we will use `np.random` to simulate. Let's generate a data set with 10,000 observations following the above mechanism

In [1]:
import numpy as np
import pandas as pd

# setting random seed for replication purposes
np.random.seed(1859031)

# setting population size
n = 10000

# creating empty DataFrame to store our generated data
df = pd.DataFrame()

# simulating W
df['W'] = np.random.binomial(n=1, p=0.5, size=n)

# simulating A
df['A'] = np.random.binomial(n=1, p=0.05 + 0.45*df['W'], size = n)

# checking simulated data
df.info()
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
W    10000 non-null int32
A    10000 non-null int32
dtypes: int32(2)
memory usage: 78.2 KB
                 W             A
count  10000.00000  10000.000000
mean       0.49780      0.274100
std        0.50002      0.446082
min        0.00000      0.000000
25%        0.00000      0.000000
50%        0.00000      0.000000
75%        1.00000      1.000000
max        1.00000      1.000000


We may want to expand our data generating mechanism to be something like the following

$$W \sim Binomial(1, 0.5)$$

$$L \sim Normal(0, 1)$$

$$A \sim Binomial(1, 0.05 + 0.15*W + 0.2*L)$$

However, you might find out that the above probability will sometimes resolve to be a negative number. This is not good for our simulation purposes. As you build more complex models, it is sometimes preferable to use a different distribution. Specifically, we can use a logistic distribution to make sure our probabilities are bounded between 0 and 1. We will use the `logistic` class from `scipy.stats` to accomplish this.

Our data generating mechanism will be a little different now 

$$W \sim Binomial(1, 0.5)$$

$$L \sim Normal(0, 1)$$

$$A \sim Binomial(1, logit(-2.94 - 1.73*W - 1.39L))$$

Let's look at this example

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import logistic

# setting random seed for replication purposes
np.random.seed(1859031)

# setting population size
n = 10000

# creating empty DataFrame to store our generated data
df = pd.DataFrame()
df['W'] = np.random.binomial(n=1, p=0.5, size=n)
df['L'] = np.random.normal(size=n)
df['A'] = np.random.binomial(n=1, p=logistic.cdf(-2.94 - 1.73*df['W'] - 1.93*df['L']), size=n)

df.info()
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
W    10000 non-null int32
L    10000 non-null float64
A    10000 non-null int32
dtypes: float64(1), int32(2)
memory usage: 156.3 KB
                 W             L             A
count  10000.00000  10000.000000  10000.000000
mean       0.49780      0.005256      0.082500
std        0.50002      1.008655      0.275139
min        0.00000     -3.785779      0.000000
25%        0.00000     -0.676611      0.000000
50%        0.00000     -0.001750      0.000000
75%        1.00000      0.695432      0.000000
max        1.00000      4.185113      1.000000


Note that the data generating mechanism we specified for $A$ should be the same parameters we get from a logistic regression model fit to our simulated data (ignoring random error). This can be a useful check for models you are using or checking that your data is simulated as intended.

## Simulating Potential Outcomes
In this next part, we will discuss simulating potential outcomes. Often we are interested in the average treatment effect, defined as 

$$ATE = \Pr(Y^{a=1}) - \Pr(Y^{a=0})$$

where $Y^{a}$ is the outcome for an individual under the treatment $A=a$. This is easy to simulate in our data. Below is our data generating mechanism

$$W \sim Binomial(1, 0.5)$$

$L \sim Normal(0, 1)$

$$A \sim Binomial(1, logit(-3 + 3*W + 1.5L))$$

$$Y^{a=1} \sim Binomial(1, logit(-3 + 2.5 + 3*W + 1.5L))$$

$$Y^{a=0} \sim Binomial(1, logit(-3 + 3*W + 1.5L))$$

To get the observed $Y$ value, we use counterfactual consistency defined as 

$$Y = Y^{a=1}*A + Y^{a=0}*(1-A)$$

Let's demonstrate

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import logistic

# setting random seed for replication purposes
np.random.seed(1859031)

# setting population size
n = 10000

# creating empty DataFrame to store our generated data
df = pd.DataFrame()
df['W'] = np.random.binomial(n=1, p=0.5, size=n)
df['L'] = np.random.normal(size=n)
df['A'] = np.random.binomial(n=1, p=logistic.cdf(-3 + 3*df['W'] - 1.5*df['L']), size=n)
# potential outcomes
df['Y0'] = np.random.binomial(n=1, p=logistic.cdf(-3 + 3*df['W'] + 1.5*df['L']), size=n)
df['Y1'] = np.random.binomial(n=1, p=logistic.cdf(-3 + 2.5 + 3*df['W'] + 1.5*df['L']), size=n)
# observed outcome
df['Y'] = df['A']*df['Y1'] + (1 - df['A'])*df['Y0']

df.info()
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
W     10000 non-null int32
L     10000 non-null float64
A     10000 non-null int32
Y0    10000 non-null int32
Y1    10000 non-null int32
Y     10000 non-null int32
dtypes: float64(1), int32(5)
memory usage: 273.5 KB
                 W             L            A            Y0            Y1  \
count  10000.00000  10000.000000  10000.00000  10000.000000  10000.000000   
mean       0.49780      0.005256      0.29450      0.300300      0.637900   
std        0.50002      1.008655      0.45584      0.458411      0.480632   
min        0.00000     -3.785779      0.00000      0.000000      0.000000   
25%        0.00000     -0.676611      0.00000      0.000000      0.000000   
50%        0.00000     -0.001750      0.00000      0.000000      1.000000   
75%        1.00000      0.695432      1.00000      1.000000      1.000000   
max        1.00000      4.185113      1.00000      1.000000  

The handy part of simulating our data in this way is that it allows us to easily calculate the true causal effect from our simulated data. We can do that through the following

In [5]:
print('ATE* =', np.mean(df['Y1'] - df['Y0']))

ATE* = 0.3376


We can also use other definitions of the average treatment effect and other distribution of variables. Below is a quick example for a Normally distributed $Y$

In [7]:
df = pd.DataFrame()
df['W'] = np.random.binomial(n=1, p=0.5, size=n)
df['L'] = np.random.normal(size=n)
df['A'] = np.random.binomial(n=1, p=logistic.cdf(-3 + 3*df['W'] - 1.5*df['L']), size=n)
# potential outcomes
df['Y0'] = 2*df['W'] + 6*df['L'] + 0.3*df['W']*df['L'] + 5*np.random.normal(size=n)
df['Y1'] = -10 + 2*df['W'] + 6*df['L'] + 0.3*df['W']*df['L'] + 5*np.random.normal(size=n)
# observed outcome
df['Y'] = df['A']*df['Y1'] + (1 - df['A'])*df['Y0']


print('ATE* =', np.mean(df['Y1'] - df['Y0']))

ATE* = -10.06959537030908


## Other Advice
Before concluding, some practical advice. First, I would wrap your simulation data mechanism in a function (demonstrated below). It makes it easier to do repeated calls and you can add a bunch of tuning parameters to change things (like the $\beta$'s or $n$). Second, always use a seed. It will keep your code consistent. Also note that the RNG seed differs across operating systems. So, you simulation code may differ between Mac, Windows, and Linux *even though you set a seed*. Third, I recommend formally writing out the data generating mechanism. I find DAGs or SWIGs to be particularly helpful for this. Lastly, start small. Start with a simulation that generates two data sets before moving to 1,000. This will allow you an opportunity to catch any errors in your simulation code

Here is an example function to simulate data sets. You can also build further complexities (like adding arguments that update parameters, but I leave that for you to do on your own)

In [8]:
def simulate_data(n, print_ate=True):
    """Function to simulate up some data
    
    Parameters
    ----------
    n : int
        Number of observations to generate
    """
    if n < 0 or not isinstance(n, int):
        raise ValueError('n must be a positive integer')
    df = pd.DataFrame()
    df['W'] = np.random.binomial(n=1, p=0.5, size=n)
    df['L'] = np.random.normal(size=n)
    df['A'] = np.random.binomial(n=1, p=logistic.cdf(-3 + 3*df['W'] - 1.5*df['L']), size=n)
    df['Y0'] = 2*df['W'] + 6*df['L'] + 0.3*df['W']*df['L'] + 5*np.random.normal(size=n)
    df['Y1'] = -10 + 2*df['W'] + 6*df['L'] + 0.3*df['W']*df['L'] + 5*np.random.normal(size=n)
    df['Y'] = df['A']*df['Y1'] + (1 - df['A'])*df['Y0']
    
    if print_ate:
        print('ATE* =', np.mean(df['Y1'] - df['Y0']))
    
    return df[['Y', 'A', 'W', 'L']]


# Conclusion
This concludes the tutorial on simulating data. Again this is a practical introduction to how to simulate data in Python. It is not meant as advice as to how to conduct a simulation study. However, the tutorial provides the framework of how to code your simulation study

I will revisit the tools described here to demonstrate some properties of Targeted-Maximum-Likelihood-Estimation later in the tutorial. For a framework on how to build a slightly more complicated simulation study, please see this tutorial

## References
Burton, A., Altman, D. G., Royston, P., & Holder, R. L. (2006). The design of simulation studies in medical statistics. SIM, 25(24), 4279-4292.